In [2]:
import os, sys
import time
from shutil import copy, rmtree
from itertools import product
import pdb
import argparse
import random
import torch
import numpy as np
from kernel.datasets import get_dataset,get_circuit_dataset,MyOwnDataset
# from kernel.train_eval import cross_validation_with_val_set
# from kernel.train_eval import cross_validation_without_val_set
from kernel.train_eval import trainFEGIN
# from kernel.gcn import *
# from kernel.graph_sage import *
# from kernel.gin import *
from kernel.FEGIN import *
# from kernel.gat import *
# from kernel.graclus import Graclus
# from kernel.top_k import TopK
# from kernel.diff_pool import *
# from kernel.global_attention import GlobalAttentionNet
# from kernel.set2set import Set2SetNet
# from kernel.sort_pool import SortPool
import warnings
import argparse
import netlsd
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from sklearn.utils import shuffle
from torch_geometric.data import DenseDataLoader as DenseLoader
from dataloader import DataLoader  # replace with custom dataloader to handle subgraphs
from torch.optim import Adam

In [3]:
device = 'mps'

# import and preprocess the dataset _____________________ generate: processsed/ltspice_examples_dataset.pt    pre_filter.pt   pre_transform.pt
dataset_name = "ltspice_examples"

dataset = MyOwnDataset("MergeTest/data/", dataset_name, 
                h = 12, # the height of rooted subgraph for NGNN models
                node_label = "spd", 
                use_rd = None, 
                max_nodes_per_hop = None)

Processing...


processing data now!
length of datalist: 7016
saving path: MergeTest/data/processed/ltspice_examples_dataset.pt


Done!


In [4]:
# porcess the dataset to be compatible with the models _____________________ generate: MergeTest/data/ltspice_examples_netlsd_train.pt    MergeTest/data/ltspice_examples_netlsd_test.pt

batch_size=128
if os.path.isfile('MergeTest/data/'+dataset_name+'_netlsd_train.pt'):
        train_des = torch.load('MergeTest/data/'+dataset_name+'_netlsd_train.pt')
        test_des = torch.load('MergeTest/data/'+dataset_name+'_netlsd_test.pt')
        train_dataset = [d for d in dataset if d.set =='train']
        test_dataset = [d for d in dataset if d.set =='test']
else:
    train_dataset,train_des, test_dataset, test_des = [],[],[],[]
    for index, d in enumerate(dataset):
        if d.set=='train':
            des = (torch.tensor(netlsd.heat(to_networkx(d, to_undirected = True)))*0.1).float()
            des_d = Data(x = des, edge_index = d.edge_index, y = d.y)
            train_dataset.append(d)
            train_des.append(des_d)
            # if index%50==0:
            #     print(index)

        else:
            des = (torch.tensor(netlsd.heat(to_networkx(d, to_undirected = True)))*0.1).float()
            des_d = Data(x = des, edge_index = d.edge_index, y = d.y)
            test_des.append(des_d)
            test_dataset.append(d)

    torch.save(train_des,'MergeTest/data/'+dataset_name+'_netlsd_train.pt')
    torch.save(test_des,'MergeTest/data/'+dataset_name+'_netlsd_test.pt')

In [5]:
 #   porcess the dataset to be compatible with the models_____________________ generate: 
train_dataset, train_des = shuffle(train_dataset, train_des)


if 'adj' in train_dataset[0]:
        train_loader = DenseLoader(train_dataset, batch_size, shuffle=False)
        test_loader = DenseLoader(test_dataset, batch_size, shuffle=False)
else:
    train_loader = DataLoader(train_dataset, batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size, shuffle=False)
    train_loader_des = DataLoader(train_des, batch_size, shuffle=False)
    test_loader_des = DataLoader(test_des, batch_size, shuffle=False)

In [7]:
 #   porcess the dataset to be compatible with the models_____________________ generate: 
Net = FEGIN
model = Net(dataset, num_layers=4, hidden=32, emb_size=250,node_label = "spd", use_rd=False)
model.to(device).reset_parameters()
model.train()
optimizer = Adam(model.parameters(), lr=1E-2, weight_decay=0)
loader = train_loader
des_loader = train_loader_des
i=0
for data,des in zip(loader,des_loader):
        optimizer.zero_grad()
        data = data.to(device)
        des = des.to(device)
        out = model(data,des)
        print(i)
        i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


In [8]:
print(out)

tensor([[-1.7266, -1.6817, -1.5722, -1.4585, -1.6303],
        [-1.7678, -1.7215, -1.5706, -1.4220, -1.6029],
        [-1.6850, -1.6950, -1.5810, -1.4856, -1.6153],
        [-1.7155, -1.6933, -1.5953, -1.4415, -1.6256],
        [-1.6984, -1.6526, -1.5710, -1.5206, -1.6142],
        [-1.7553, -1.6697, -1.5989, -1.3848, -1.6801],
        [-1.7459, -1.6599, -1.5845, -1.4599, -1.6192],
        [-1.7991, -1.7069, -1.5228, -1.4666, -1.5882],
        [-1.7086, -1.6907, -1.5636, -1.4658, -1.6388],
        [-1.7674, -1.6996, -1.5180, -1.4518, -1.6442],
        [-1.6514, -1.7010, -1.5410, -1.5144, -1.6523],
        [-1.6761, -1.6723, -1.5777, -1.4978, -1.6348],
        [-1.7155, -1.7011, -1.5722, -1.4613, -1.6188],
        [-1.7219, -1.7160, -1.5941, -1.4378, -1.6046],
        [-1.7211, -1.7362, -1.5713, -1.4369, -1.6119]], device='mps:0',
       grad_fn=<LogSoftmaxBackward0>)


In [9]:
print(out.size())

torch.Size([15, 5])
